# 하나은행 환율 정보 수집

### 1) 1페이지 자료 추출해서 데이터 프레임으로 출력하기

In [14]:
# selenium 4
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs
from datetime import date
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time
import pandas as pd
import dbio
import warnings
warnings.simplefilter(action = 'ignore', category = FutureWarning)

In [ ]:
# ! pip 

In [12]:
date = "2024-07-22"
#크롬 옵션즈에 정보를 담아 사람인 것 처럼 만들기
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ['enable-logging'])#웹브라웆저 콘솔에 로그인 찍히는 것 줄여준다
options.add_argument("Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36")
options.add_argument('lang=ko_KR')

#크롬 웹브라우저 드라이버 자동 다운로드
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
driver.set_window_size(1920, 1080)   #웹브라우저 해상도 조절
driver.get("https://www.kebhana.com/cms/rate/index.do?contentUrl=/cms/rate/wpfxd651_01i.do#//HanaBank")  #개발자모드에서 찾은 링크로 접속
time.sleep(6)
page_html = driver.page_source
df = pd.read_html(page_html)   #-> 표읽으려고 시도하려했지만 읽지못함. 데이터가 오지 않는다 #판다스로 읽어올 수 가 없다

In [ ]:
# 하나은행 링크 : https://www.hanabank.com/cms/rate/index.do?contentUrl=/cms/rate/wpfxd651_01i.do#//HanaBank

In [ ]:
#날짜입력창
serch_box = driver.find_element(By.CSS_SELECTOR, '#tmpInqStrDt') #아이디가 있으면 굳이 길게 안써줘도 가능
serch_box.clear()   #값을 넣어주려면 먼저 입력된 값을 지워줘야 된다.
serch_box.send_keys(date +Keys.ENTER)  

# 조회버튼
search_button=  driver.find_element(By.CSS_SELECTOR, "#HANA_CONTENTS_DIV > div.btnBoxCenter > a")
search_button.click()

page_html = driver.page_source
df =pd.read_html(page_html)[1]
dbio.exi_to_db("exchange_rate", date, df)


#db에 저장
#딕셔너리에 새로운 키 넣는법 -> df['새로운 컬럼명']=새로운 값
#딕셔너리랑 데이터프레임 사용법 같다
df['date'] = date

In [ ]:
df.columns

In [2]:
!pip install selenium
!pip install webdrivermanager 

In [ ]:
df

In [11]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup as bs
from datetime import date
import pandas as pd
import dbio
import time


In [12]:
today =str(date.today())
#위에 있는거 합쳐서 한번에
#크롬 옵션즈에 정보를 담아 사람인 것 처럼 만들기
options = webdriver.ChromeOptions()
# options.add_experimental_option("excludeSwitches", ['enable-logging'])
# options.add_argument("Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36")
# options.add_argument('lang=ko_KR')
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')


#크롬 웹브라우저 드라이버 자동 다운로드
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options= options)
driver.set_window_size(1920, 1080) # 웹브라우저 해상도 조절
driver.get("https://www.kebhana.com/cms/rate/index.do?contentUrl=/cms/rate/wpfxd651_01i.do#//HanaBank")
# 웹 요소가 나타날 때까지 최대 10초 동안 기다립니다.
wait = WebDriverWait(driver, 10)

#날짜 변경
# search_box = driver.find_element(By.CSS_SELECTOR, "#tmpInqStrDt") # 얘는 id이므로 유일함 무조건 저거 하나만 쳐도 나옴
search_box = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '#tmpInqStrDt')))
search_box.clear()
search_box.send_keys(today+Keys.ENTER)

#조회하기 버튼
search_button = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#HANA_CONTENTS_DIV > div.btnBoxCenter > a")))
search_button.click()

#환율 정보 테이블이 뜰 때까지 기다림
exchange_rate_table = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '#searchContentDiv > div.printdiv > table')))


page_html = driver.page_source
df = pd.read_html(page_html)[1]
df['date'] = today
cols = ('통화', '현찰사실때환율', '현찰살때Spread', '현찰팔때환율', '현찰팔때Spread', '송금보낼때', '송금받을때', 'T/C살때', '외화수표팔때', '매매기준율', '환가_료율', '미화환산율', 'date')
sorted_cols = ('date','통화', '현찰사실때환율', '현찰살때Spread', '현찰팔때환율', '현찰팔때Spread', '송금보낼때', '송금받을때', 'T/C살때', '외화수표팔때', '매매기준율', '환가_료율', '미화환산율')
df.columns = cols
df = df[[*sorted_cols]]


#db에 저장
dbio.exi_to_db("exchange_rate", today, df)

C:\Users\5-06\AppData\Local\Temp\ipykernel_15084\3691589575.py:35: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(page_html)[1]


# 날짜를 만들어서 오늘 전 것까지 크롤링
* 날짜 리스트 생성하기

In [14]:
import pandas as pd

# 시작 날짜와 끝 날짜 설정
start_date = '1995-01-03'
end_date = '2024-07-31'

# 주중(월~금) 날짜만 포함하는 날짜 범위 생성
date_range = pd.date_range(start=start_date, end=end_date, freq='B')  # 'B'는 Business day (월~금)을 나타냅니다.

# 날짜를 문자열 리스트로 변환
dates_list = date_range.strftime('%Y-%m-%d').tolist()

# 결과 출력 (리스트)
print(dates_list)


['1995-01-03', '1995-01-04', '1995-01-05', '1995-01-06', '1995-01-09', '1995-01-10', '1995-01-11', '1995-01-12', '1995-01-13', '1995-01-16', '1995-01-17', '1995-01-18', '1995-01-19', '1995-01-20', '1995-01-23', '1995-01-24', '1995-01-25', '1995-01-26', '1995-01-27', '1995-01-30', '1995-01-31', '1995-02-01', '1995-02-02', '1995-02-03', '1995-02-06', '1995-02-07', '1995-02-08', '1995-02-09', '1995-02-10', '1995-02-13', '1995-02-14', '1995-02-15', '1995-02-16', '1995-02-17', '1995-02-20', '1995-02-21', '1995-02-22', '1995-02-23', '1995-02-24', '1995-02-27', '1995-02-28', '1995-03-01', '1995-03-02', '1995-03-03', '1995-03-06', '1995-03-07', '1995-03-08', '1995-03-09', '1995-03-10', '1995-03-13', '1995-03-14', '1995-03-15', '1995-03-16', '1995-03-17', '1995-03-20', '1995-03-21', '1995-03-22', '1995-03-23', '1995-03-24', '1995-03-27', '1995-03-28', '1995-03-29', '1995-03-30', '1995-03-31', '1995-04-03', '1995-04-04', '1995-04-05', '1995-04-06', '1995-04-07', '1995-04-10', '1995-04-11', '1995

In [15]:
len(dates_list)

7717

In [16]:
str(dates_list[0])

'1995-01-03'

In [18]:
for weekday in dates_list[:5]:
    print(str(weekday))

1995-01-03
1995-01-04
1995-01-05
1995-01-06
1995-01-09


# 1995년 1월 3일부터 2024년 7월 31일까지 평일 환율 정보 수집

In [21]:
# selenium 4
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs
from datetime import date
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time
import pandas as pd
import dbio
import warnings
warnings.simplefilter(action = 'ignore', category = FutureWarning)

for weekday in dates_list:
    today =str(weekday)
    #위에 있는거 합쳐서 한번에
    #크롬 옵션즈에 정보를 담아 사람인 것 처럼 만들기
    options = webdriver.ChromeOptions()
    # options.add_experimental_option("excludeSwitches", ['enable-logging'])
    # options.add_argument("Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36")
    # options.add_argument('lang=ko_KR')
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')


    #크롬 웹브라우저 드라이버 자동 다운로드
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options= options)
    driver.set_window_size(1920, 1080) # 웹브라우저 해상도 조절
    driver.get("https://www.kebhana.com/cms/rate/index.do?contentUrl=/cms/rate/wpfxd651_01i.do#//HanaBank")
    # 웹 요소가 나타날 때까지 최대 10초 동안 기다립니다.
    wait = WebDriverWait(driver, 10)

    #날짜 변경
    # search_box = driver.find_element(By.CSS_SELECTOR, "#tmpInqStrDt") # 얘는 id이므로 유일함 무조건 저거 하나만 쳐도 나옴
    search_box = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '#tmpInqStrDt')))
    search_box.clear()
    search_box.send_keys(today+Keys.ENTER)

    #조회하기 버튼
    search_button = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#HANA_CONTENTS_DIV > div.btnBoxCenter > a")))
    search_button.click()

    #환율 정보 테이블이 뜰 때까지 기다림
    exchange_rate_table = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '#searchContentDiv > div.printdiv > table')))


    page_html = driver.page_source
    df = pd.read_html(page_html)[1]
    df['date'] = today
    cols = ('통화', '현찰사실때환율', '현찰살때Spread', '현찰팔때환율', '현찰팔때Spread', '송금보낼때', '송금받을때', 'T/C살때', '외화수표팔때', '매매기준율', '환가_료율', '미화환산율', 'date')
    sorted_cols = ('date','통화', '현찰사실때환율', '현찰살때Spread', '현찰팔때환율', '현찰팔때Spread', '송금보낼때', '송금받을때', 'T/C살때', '외화수표팔때', '매매기준율', '환가_료율', '미화환산율')
    df.columns = cols
    df = df[[*sorted_cols]]


    #db에 저장
    dbio.exi_to_db("exchange_rate", today, df)

In [13]:
#===============================================================================

In [26]:
len(df)

2

In [27]:
df[1]

통화       현찰                              송금          T/C 사실때  \
                 통화     사실 때            파실 때            보낼 때     받을 때 T/C 사실때   
                 통화       환율 Spread       환율 Spread     보낼 때     받을 때 T/C 사실때   
0            미국 USD  1386.44   1.75  1338.76   1.75  1375.90  1349.30     0.0   
1      일본 JPY (100)   925.34   1.75   893.52   1.75   918.34   900.52     0.0   
2            유로 EUR  1504.57   1.99  1445.87   1.99  1489.97  1460.47     0.0   
3            중국 CNY   197.80   5.00   178.98   5.00   190.27   186.51     0.0   
4            홍콩 HKD   177.83   1.97   170.97   1.97   176.14   172.66     0.0   
5            태국 THB    40.35   5.00    36.13   6.00    38.81    38.05     0.0   
6            대만 TWD    47.12  13.10    37.51  10.00     0.00     0.00     0.0   
7           필리핀 PHP    25.70  10.00    21.46   8.20    23.60    23.14     0.0   
8          싱가포르 SGD  1040.31   1.99   999.73   1.99  1030.22  1009.82     0.0   
9            호주 AUD   907.23   1.97   872.19   1.97   898.60   880.82     0.0   
10    베트남 VND (100)     6.04  11.80     4.78  11.80     5.46     5.36     0.0   
11           영국 GBP  1785.22   1.97  1716.26   1.97  1768.24  1733.24     0.0   
12          캐나다 CAD  1006.37   1.97   967.49   1.97   996.79   977.07     0.0   
13        말레이시아 MYR   321.13   7.40   276.89   7.40     0.00   296.02     0.0   
14          러시아 RUB    17.31   9.50    13.05  17.50    16.36    15.26     0.0   
15        남아공화국 ZAR    80.67   8.00    68.73   8.00    75.59    73.81     0.0   
16         노르웨이 NOK   129.07   3.30   117.83   5.70   126.19   123.71     0.0   
17         뉴질랜드 NZD   826.65   1.97   794.71   1.97   818.78   802.58     0.0   
18          덴마크 DKK   204.20   3.30   186.42   5.70   199.65   195.71     0.0   
19          멕시코 MXN    80.64  10.50    65.32  10.50    73.70    72.26     0.0   
20           몽골 MNT     0.00   0.00     0.00   0.00     0.00     0.00     0.0   
21          바레인 BHD  3845.84   6.40  3325.36   8.00  3650.66  3578.38     0.0   
22        방글라데시 BDT     0.00   0.00     0.00   0.00     0.00     0.00     0.0   
23          브라질 BRL   265.39  10.20   216.75  10.00     0.00   237.95     0.0   
24         브루나이 BND  1060.82   4.00   958.82   6.00     0.00     0.00     0.0   
25      사우디아라비아 SAR   386.06   6.30   338.13   6.90   366.81   359.55     0.0   
26         스리랑카 LKR     0.00   0.00     0.00   0.00     0.00     0.00     0.0   
27          스웨덴 SEK   131.46   3.30   120.02   5.70   128.54   126.00     0.0   
28          스위스 CHF  1583.76   1.97  1522.58   1.97  1568.70  1537.64     0.0   
29    아랍에미리트공화국 AED   391.38   5.50   345.39   6.90   374.68   367.28     0.0   
30          알제리 DZD     0.00   0.00     0.00   0.00     0.00     0.00     0.0   
31           오만 OMR  3786.86   7.00  3291.40   7.00     0.00     0.00     0.0   
32          요르단 JOD  2092.90   8.90  1768.12   8.00     0.00     0.00     0.0   
33         이스라엘 ILS   395.10  10.00   330.46   8.00     0.00     0.00     0.0   
34          이집트 EGP     0.00   0.00     0.00   0.00     0.00     0.00     0.0   
35           인도 INR     0.00   0.00     0.00   0.00     0.00     0.00     0.0   
36  인도네시아 IDR (100)     9.26  10.00     7.58  10.00     8.50     8.34     0.0   
37           체코 CZK    62.96   8.50    52.81   9.00    58.66    57.40     0.0   
38           칠레 CLP     1.59  10.00     1.31  10.00     0.00     0.00     0.0   
39        카자흐스탄 KZT     0.00   0.00     0.00   0.00     0.00     0.00     0.0   
40          카타르 QAR     0.00   0.00     0.00   0.00     0.00     0.00     0.0   
41           케냐 KES     0.00   0.00     0.00   0.00     0.00     0.00     0.0   
42         콜롬비아 COP     0.00   0.00     0.00   0.00     0.00     0.00     0.0   
43         쿠웨이트 KWD  4749.67   6.50  4103.01   8.00  4504.38  4415.20     0.0   
44         탄자니아 TZS     0.00   0.00     0.00   0.00     0.00     0.00     0.0   
45         튀르키예 TRY     0.00   0.00     0.00   0.00    41.52    40.62     0.0   
46         파키스탄 PKR     0.00   0.00     0.00   0

In [ ]:
# 모듈로 만들어서 쉽게 부러오자. 매번 복붙 하지 말고.

In [ ]:
def db_connect():
    engine = create_engine("%s+%s://%s:%s@%s/%s" % (db, dbtype, id, pw, host, database))  #컴파일 과정에서 인식이 안되는 것 같다. 다 서식 지정자로 바꿈 -> 오직 이것만 성공
    conn = engine.connect()
    return conn



# DB에 접속해서 저장하는 함수
def stock_info_to_db(idx, code, df):
    today = str(date.today()).replace("-","_")
    conn = db_connect()
    df.to_sql(f"{today[:7]}_stock_price_info", con=conn, if_exists='append', index=False)
    conn.close()
    
    return print(f"{today}, {idx}, {code}, {'저장완료':<30s}", end="\r")

# 1. 30년치 저장하기
* 1) 1페이지 자료 추출해서 데이터 프레임으로 출력하기
* 2) 과거 환율자료 1995.01.03부터 어제까지 (주5일 가능하다면해보기)(윤달도 적용)
* (2024-1995)*365*5초/3600=14.7시간
* 3) DB에 그때그때 저장되도록하기

# 2. 오늘부터 외환시장 장 마감 후 데이터 수집 자동화
* 1) 1페이지 자료 추출해서 데이터프레임으로 출력하기
* 2) DB에 가져올 때마다 저장하기
* 3) 실행파일로 만들기
* 4) 스케줄러에 등록

#### 이번 주 마무리는 환율정보 쌓기
* 경제는 정치/사회 문제에 큰 영향

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup as bs
import time
import random
import lxml
import pandas as pd

In [3]:
# 크롬 옵션즈에 User-Agent, lang 같은 정보를 담아 셀레니움을 이용한 크롤링이 아닌 것 처럼 만들기
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-logging"])
options.add_argument('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36')
options.add_argument("lang=ko_KR")

# 웹드라이버를 버전에 맞게 자동으로 다운 받고 옵션을 추가해 줌
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
driver.set_window_size(1920,1080)   # 웹브라우저 해상도 조절

driver.get("https://www.hanabank.com/cms/rate/index.do?contentUrl=/cms/rate/wpfxd651_01i.do#//HanaBank")  # 크롤링 할 웹사이트 주소

In [4]:
# 날짜 박스 #tmpInqStrDt
serch_box = driver.find_element(By.CSS_SELECTOR, '#tmpInqStrDt')
serch_box.clear()   # 기존에 입력된 값을 지웁니다.
serch_box.send_keys("2024-07-30") #+Keys.ENTER)


# 조회버튼   #HANA_CONTENTS_DIV > div.btnBoxCenter > a
search_box = driver.find_element(By.CSS_SELECTOR, '#HANA_CONTENTS_DIV > div.btnBoxCenter > a')
search_box.click()   # 조회버튼 클릭

In [17]:
# from datetime import datetime
# from dateutil.relativedelta import relativedelta

# # 오늘 날짜를 가져옵니다.
# today = datetime.today()
# # 10년 전의 날짜를 계산합니다.
# ten_years_ago = today - relativedelta(years=10)

# # 날짜 범위를 생성합니다.
# dates = [ten_years_ago + relativedelta(days=i) for i in range((today - ten_years_ago).days)]

# # 결과를 출력합니다.
# for date in dates[::-1][:10]:
#     print(date.strftime('%Y-%m-%d'))

2024-07-31
2024-07-30
2024-07-29
2024-07-28
2024-07-27
2024-07-26
2024-07-25
2024-07-24
2024-07-23
2024-07-22


In [5]:
page_html = driver.page_source

In [6]:
page_html

'<html xmlns="http://www.w3.org/1999/xhtml" lang="ko" xml:lang="ko"><head>\n<title>하나은행 개인뱅킹</title>\n<meta http-equiv="Content-Type" content="text/html; charset=utf-8">\n<meta http-equiv="Cache-control" content="No-cache">\n<meta http-equiv="Expires" content="0">\n<meta http-equiv="Pragma" content="No-cache">\n<meta name="Description" content="">\n<meta name="Keywords" content="">\n<meta http-equiv="X-UA-Compatible" content="IE=edge">\n\n<link rel="shortcut icon" href="/favicon.ico">\n<script async="" charset="utf-8" src="https://logcol.kebhana.com:8443/static/96580/ntm.js?ver=1722470400000"></script><script async="" charset="utf-8" src="https://logcol.kebhana.com:8443/static/96580/install.js?ver=1722470400000"></script><script type="text/javascript">\nvar TNK_SR = \'0915e83a3a0a024a66a9c821184d5413\';\nvar _W_L_M_ = \'P\';\nvar DOMAIN_HOST_NM = \'pbk.prdhostNm\';\nvar isTransKey       = false;  // 마우스입력기 사용여부\n\n \nwindow._P1_CAUTH_LOG = \'FpyIkPDeNiXqQmwuUAKE9A==:390c4efa-f942-4542-

In [7]:
soup = bs(page_html,'lxml')
money_list = soup.select("#searchContentDiv > div.printdiv > table > tbody > tr")
money_list

[<tr>
 <td class="tc">
 <a href="#//HanaBank" onclick="pbk.foreign.rate.pbld.prs.goFluctuation('USD','20240730');" title="미국 USD 일일변동내역 보기">
 <u>
 							
 							
 							미국 USD
 							
 							</u>
 </a>
 </td>
 <td class="txtAr">1,409.23</td>
 <td class="txtAr">1.75</td>
 <td class="txtAr">1,360.77</td>
 <td class="txtAr">1.75</td>
 <td class="txtAr">1,398.50</td>
 <td class="txtAr">1,371.50</td>
 <td class="txtAr">0.00</td>
 <td class="txtAr">1,368.74</td>
 <td class="txtAr">1,385.00</td>
 <td class="txtAr">7.19371</td>
 <td class="txtAr">1.0000</td>
 </tr>,
 <tr>
 <td class="tc">
 <a href="#//HanaBank" onclick="pbk.foreign.rate.pbld.prs.goFluctuation('JPY','20240730');" title="일본 JPY (100) 일일변동내역 보기">
 <u>
 							
 							
 							일본 JPY (100)
 							
 							</u>
 </a>
 </td>
 <td class="txtAr">921.73</td>
 <td class="txtAr">1.75</td>
 <td class="txtAr">890.03</td>
 <td class="txtAr">1.75</td>
 <td class="txtAr">914.75</td>
 <td class="txtAr">897.01</td>
 <td class="txtAr">0.0

In [39]:
money_list[0]

<tr>
<td class="tc">
<a href="#//HanaBank" onclick="pbk.foreign.rate.pbld.prs.goFluctuation('USD','20240730');" title="미국 USD 일일변동내역 보기">
<u>
							
							
							미국 USD
							
							</u>
</a>
</td>
<td class="txtAr">1,409.23</td>
<td class="txtAr">1.75</td>
<td class="txtAr">1,360.77</td>
<td class="txtAr">1.75</td>
<td class="txtAr">1,398.50</td>
<td class="txtAr">1,371.50</td>
<td class="txtAr">0.00</td>
<td class="txtAr">1,368.74</td>
<td class="txtAr">1,385.00</td>
<td class="txtAr">7.19371</td>
<td class="txtAr">1.0000</td>
</tr>

In [58]:
money_list[0].select_one("td>a>u").text.replace("\n","").replace("\t","")

'미국 USD'

In [60]:
money_list[0].select_one("td:nth-child(2)").text

'1,409.23'

In [62]:
for bank in money_list:
    title = bank.select_one("td>a>u").text.replace("\n","").replace("\t","") # 책 제목
    author = money_list[0].select_one("td:nth-child(2)").text # 저자
    publisher = money_list[0].select_one("td:nth-child(3)").text 
    pub_date = money_list[0].select_one("td:nth-child(4)").text 
    pub_date = money_list[0].select_one("td:nth-child(5)").text
    pub_date = money_list[0].select_one("td:nth-child(6)").text
    pub_date = money_list[0].select_one("td:nth-child(7)").text
    pub_date = money_list[0].select_one("td:nth-child(8)").text
    pub_date = money_list[0].select_one("td:nth-child(9)").text
    pub_date = money_list[0].select_one("td:nth-child(10)").text
    pub_date = money_list[0].select_one("td:nth-child(11)").text
    pub_date = money_list[0].select_one("td:nth-child(12)").text
    
    
    print(title, author, publisher, pub_date)

미국 USD 1,409.23 1.75 1.0000
일본 JPY (100) 1,409.23 1.75 1.0000
유로 EUR 1,409.23 1.75 1.0000
중국 CNY 1,409.23 1.75 1.0000
홍콩 HKD 1,409.23 1.75 1.0000
태국 THB 1,409.23 1.75 1.0000
대만 TWD 1,409.23 1.75 1.0000
필리핀 PHP 1,409.23 1.75 1.0000
싱가포르 SGD 1,409.23 1.75 1.0000
호주 AUD 1,409.23 1.75 1.0000
베트남 VND (100) 1,409.23 1.75 1.0000
영국 GBP 1,409.23 1.75 1.0000
캐나다 CAD 1,409.23 1.75 1.0000
말레이시아 MYR 1,409.23 1.75 1.0000
러시아 RUB 1,409.23 1.75 1.0000
남아공화국 ZAR 1,409.23 1.75 1.0000
노르웨이 NOK 1,409.23 1.75 1.0000
뉴질랜드 NZD 1,409.23 1.75 1.0000
덴마크 DKK 1,409.23 1.75 1.0000
멕시코 MXN 1,409.23 1.75 1.0000
몽골 MNT 1,409.23 1.75 1.0000
바레인 BHD 1,409.23 1.75 1.0000
방글라데시 BDT 1,409.23 1.75 1.0000
브라질 BRL 1,409.23 1.75 1.0000
브루나이 BND 1,409.23 1.75 1.0000
사우디아라비아 SAR 1,409.23 1.75 1.0000
스리랑카 LKR 1,409.23 1.75 1.0000
스웨덴 SEK 1,409.23 1.75 1.0000
스위스 CHF 1,409.23 1.75 1.0000
아랍에미리트공화국 AED 1,409.23 1.75 1.0000
알제리 DZD 1,409.23 1.75 1.0000
오만 OMR 1,409.23 1.75 1.0000
요르단 JOD 1,409.23 1.75 1.0000
이스라엘 ILS 1,409.23 1.7

In [ ]:
def bankinfo_to_db(df):
    from sqlalchemy import create_engine
    import pymysql
    from dbenv2 import db, dbtype, id, pw, host, database
    from datetime import date
    pymysql.install_as_MySQLdb()

    today = str(date.today()).replace("-","")

    engine = create_engine(f"{os.getenv('db')}+{os.getenv('dbtype')}://{os.getenv('id')}:{os.getenv('pw')}@{os.getenv('host')}/{os.getenv('database')}")
    conn = engine.connect()
    df.to_sql(f"bank_info", con=conn, if_exists='append', index=False)
    conn.close()

    return print(f"{'저장완료':<30s}", end="\r")
bank_info_to_db(df)

In [8]:
df = pd.read_html(page_html)[1]
df

C:\Users\5-06\AppData\Local\Temp\ipykernel_9056\3837766736.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(page_html)[1]


통화       현찰                              송금          T/C 사실때  \
                 통화     사실 때            파실 때            보낼 때     받을 때 T/C 사실때   
                 통화       환율 Spread       환율 Spread     보낼 때     받을 때 T/C 사실때   
0            미국 USD  1409.23   1.75  1360.77   1.75  1398.50  1371.50     0.0   
1      일본 JPY (100)   921.73   1.75   890.03   1.75   914.75   897.01     0.0   
2            유로 EUR  1527.47   1.99  1467.87   1.99  1512.64  1482.70     0.0   
3            중국 CNY   200.76   5.00   181.64   5.00   193.11   189.29     0.0   
4            홍콩 HKD   180.76   1.97   173.78   1.97   179.04   175.50     0.0   
5            태국 THB    40.54   5.00    36.30   6.00    38.99    38.23     0.0   
6            대만 TWD    47.70  13.10    37.97  10.00     0.00     0.00     0.0   
7           필리핀 PHP    26.00  10.00    21.71   8.20    23.87    23.41     0.0   
8          싱가포르 SGD  1052.49   1.99  1011.43   1.99  1042.27  1021.65     0.0   
9            호주 AUD   923.34   1.97   887.68   1.97   914.56   896.46     0.0   
10    베트남 VND (100)     6.12  11.80     4.84  11.80     5.53     5.43     0.0   
11           영국 GBP  1812.66   1.97  1742.64   1.97  1795.42  1759.88     0.0   
12          캐나다 CAD  1019.44   1.97   980.06   1.97  1009.74   989.76     0.0   
13        말레이시아 MYR   321.79   7.40   277.45   7.40     0.00   296.63     0.0   
14          러시아 RUB    17.68   9.50    13.33  17.50    16.71    15.59     0.0   
15        남아공화국 ZAR    81.61   8.00    69.53   8.00    76.47    74.67     0.0   
16         노르웨이 NOK   130.56   3.30   119.19   5.70   127.65   125.13     0.0   
17         뉴질랜드 NZD   833.60   1.97   801.40   1.97   825.67   809.33     0.0   
18          덴마크 DKK   207.32   3.30   189.27   5.70   202.70   198.70     0.0   
19          멕시코 MXN    81.57  10.50    66.07  10.50    74.55    73.09     0.0   
20           몽골 MNT     0.00   0.00     0.00   0.00     0.00     0.00     0.0   
21          바레인 BHD  3909.27   6.40  3380.20   8.00  3710.87  3637.39     0.0   
22        방글라데시 BDT     0.00   0.00     0.00   0.00     0.00     0.00     0.0   
23          브라질 BRL   271.78  10.20   221.97  10.00     0.00   243.68     0.0   
24         브루나이 BND  1073.23   4.00   970.05   6.00     0.00     0.00     0.0   
25      사우디아라비아 SAR   392.43   6.30   343.71   6.90   372.87   365.49     0.0   
26         스리랑카 LKR     0.00   0.00     0.00   0.00     0.00     0.00     0.0   
27          스웨덴 SEK   132.84   3.30   121.27   5.70   129.88   127.32     0.0   
28          스위스 CHF  1599.51   1.97  1537.71   1.97  1584.29  1552.93     0.0   
29    아랍에미리트공화국 AED   397.81   5.50   351.07   6.90   380.85   373.31     0.0   
30          알제리 DZD     0.00   0.00     0.00   0.00     0.00     0.00     0.0   
31           오만 OMR  3849.12   7.00  3345.50   7.00     0.00     0.00     0.0   
32          요르단 JOD  2127.31   8.90  1797.19   8.00     0.00     0.00     0.0   
33         이스라엘 ILS   406.92  10.00   340.34   8.00     0.00     0.00     0.0   
34          이집트 EGP     0.00   0.00     0.00   0.00     0.00     0.00     0.0   
35           인도 INR     0.00   0.00     0.00   0.00     0.00     0.00     0.0   
36  인도네시아 IDR (100)     9.37  10.00     7.67  10.00     8.60     8.44     0.0   
37           체코 CZK    63.91   8.50    53.61   9.00    59.55    58.27     0.0   
38           칠레 CLP     1.59  10.00     1.31  10.00     0.00     0.00     0.0   
39        카자흐스탄 KZT     0.00   0.00     0.00   0.00     0.00     0.00     0.0   
40          카타르 QAR     0.00   0.00     0.00   0.00     0.00     0.00     0.0   
41           케냐 KES     0.00   0.00     0.00   0.00     0.00     0.00     0.0   
42         콜롬비아 COP     0.00   0.00     0.00   0.00     0.00     0.00     0.0   
43         쿠웨이트 KWD  4826.81   6.50  4169.65   8.00  4577.54  4486.90     0.0   
44         탄자니아 TZS     0.00   0.00     0.00   0.00     0.00     0.00     0.0   
45         튀르키예 TRY     0.00   0.00     0.00   0.00    42.33    41.41     0.0   
46         파키스탄 PKR     0.00   0.00     0.00   0

In [10]:
import pandas as pd
from sqlalchemy import create_engine, text
import pymysql
from dbenv2 import db, dbtype, id, pw, host, database


#pymysql을 MySQLdb로 설치
pymysql.install_as_MySQLdb()

# 데이터베이스 엔진 생성
engine = create_engine("%s+%s://%s:%s@%s" % (db, dbtype, id, pw, host))
conn = engine.connect()

# 데이터베이스가 없으면 생성
conn.execute(text('CREATE DATABASE IF NOT EXISTS hana_bank_info'))
conn.close()

# DB에 접속해서 저장하는 함수
def bank_info_to_db(df):
    from sqlalchemy import create_engine
    import pymysql
    from dbenv2 import db, dbtype, id, pw, host, database
    from datetime import date
    pymysql.install_as_MySQLdb()

    today = str(date.today()).replace("-","")

    engine = create_engine("%s+%s://%s:%s@%s/%s" % (db, dbtype, id, pw, host, database))
    conn = engine.connect()
    df.to_sql(f"bank_info", con=conn, if_exists='append', index=False)
    conn.close()

    return print(f"{'저장완료':<30s}", end="\r")

bank_info_to_db(df)